In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
import os

# To plot pretty figures
#%matplotlib inline
# import matplotlib as mpl
import matplotlib.pyplot as plt
# mpl.rc('axes', labelsize=14)
# mpl.rc('xtick', labelsize=12)
# mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "lab11"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


def get_path(fig_name, fig_extension="png"):
    path = os.path.join(IMAGES_PATH, fig_name + "." + fig_extension)
    return path


In [2]:
from qiskit import QuantumCircuit, transpile, QuantumRegister,ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector
from qiskit.circuit.library import UnitaryGate
from qiskit.circuit.library import MCXGate
import numpy as np

# BB84

In [3]:
np.random.seed(seed=0)
n = 100 
# Step1 - Alice generates her randoms set of bits
alice_bits = np.random.randint(2, size=n)
print(f"Alice bits {alice_bits}") # only known to Alice

Alice bits [0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 0
 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0
 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 0]


In [4]:
#Step 2, alice choose to encode each bit on qubit in the X or Z basis at random and stores the choice for each qubit in alice_bases. 0=prepare in Z-basis, 1=prepare in X basis
alice_bases = np.random.randint(2, size=n)
print(f"Alice bases {alice_bases}")

Alice bases [1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 0
 0 0 0 1 1 0 0 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0]


In [5]:
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # prepare in Z-basis
            if bits[i] == 0: 
                pass
            else:
                qc.x(0)
        else: #prepare in X basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message        

In [6]:
message = encode_message(alice_bits, alice_bases)
print(f"Bit {alice_bits[0]} is encoded in base {alice_bases[0]} and the circuit is:")
message[0].draw()

Bit 0 is encoded in base 1 and the circuit is:


┌───┐ ░ 
  q: ┤ H ├─░─
     └───┘ ░ 
c: 1/════════

In [7]:
#Step 3, bob receives the message and measures each qubit in the X or Z basis, at random and stores this information
bob_bases = np.random.randint(2, size=n)

In [8]:
def measure_message(message, bases):
    measurements = []
    for q in range(n):
        if bases[q] == 0: #measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in the X-basis
            message[q].h(0)
            message[q].measure(0,0)
        simulator = AerSimulator()
        qcirc = transpile(message[q], simulator)
        result = simulator.run(qcirc,shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [9]:
bob_results = measure_message(message, bob_bases)
message[0].draw()
# Bob keeps the results private:
print(bob_results)

[0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0]


In [10]:
# Step 4
# Alice reveals wich qubits where encoded in which basis, basically she shares the base, and Bob do the sames.
# Now, both having informations regarding the basis, they will remove the garbage qubits that don't match
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            good_bits.append(int(bits[q]))
    return good_bits
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
print(f"Alice's key: {alice_key}")
bob_key = remove_garbage(alice_bases, bob_bases,bob_results)
print(f"Bob's key: {bob_key}")

Alice's key: [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0]
Bob's key: [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0]


In [11]:
# Step 5 - Alice and Bob compare a random selection of the bits in their keys to make sure the protocol has worked perfectly
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the bit we sample is always in the list range
        i = np.mod(i, len(bits))
        #pop removes the element of the list at index i
        sample.append(bits.pop(i))
    return sample

sample_size = 15
bit_selection = np.random.randint(n, size=sample_size)

bob_sample = sample_bits(bob_key, bit_selection)
print("  bob_sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("alice_sample = "+ str(alice_sample))

print(bob_key)
print(alice_key)
print("key length = %i" % len(alice_key))

  bob_sample = [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
alice_sample = [0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0]
key length = 33


# B92

In [12]:
np.random.seed(seed=0)
n = 100 
# Step1 - Alice generates her randoms set of bits
alice_bits = np.random.randint(2, size=n)
print(f"Alice bits {alice_bits}") # only known to Alice

Alice bits [0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 0
 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0
 0 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 1 0 0 1 1 0]


In [13]:
def b92_encode_message(bits):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bits[i] == 0:
            pass
        else:
            qc.h(0)
        qc.barrier()
        message.append(qc)
    return message        



In [14]:
message = b92_encode_message(alice_bits)
print(f"Bit {alice_bits[0]} the circuit is:")
message[0].draw()

Bit 0 the circuit is:


░ 
  q: ─░─
      ░ 
c: 1/═══

In [15]:
bob_bases = np.random.randint(2, size=n)

In [16]:
def b92_measure_message(message, bases):
    measurements = []
    for q in range(n):
        if bases[q] == 0: #measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in the X-basis
            message[q].h(0)
            message[q].measure(0,0)
        simulator = AerSimulator()
        qcirc = transpile(message[q], simulator)
        result = simulator.run(qcirc,shots=1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [17]:
bob_results = b92_measure_message(message, bob_bases)
message[0].draw()
# Bob keeps the results private:
print(bob_results)

[1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]


In [18]:
# Step 4
# Alice reveals wich qubits where encoded in which basis, basically she shares the base, and Bob do the sames.
# Now, both having informations regarding the basis, they will remove the garbage qubits that don't match
def b92_remove_garbage(a_bases, bits):
    good_bits = []
    for q in range(n):
        if bits[q]==1:
            good_bits.append(int(a_bases[q]))
    return good_bits
alice_key = b92_remove_garbage(alice_bases, bob_results)
print(f"Alice's key: {alice_key}")
bob_key = b92_remove_garbage(bob_bases,bob_results)
print(f"Bob's key: {bob_key}")

Alice's key: [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
Bob's key: [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]


In [19]:
# Step 5 - Alice and Bob compare a random selection of the bits in their keys to make sure the protocol has worked perfectly
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the bit we sample is always in the list range
        i = np.mod(i, len(bits))
        #pop removes the element of the list at index i
        sample.append(bits.pop(i))
    return sample

sample_size = 15
bit_selection = np.random.randint(n, size=sample_size)

bob_sample = sample_bits(bob_key, bit_selection)
print("  bob_sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("alice_sample = "+ str(alice_sample))

print(bob_key)
print(alice_key)
print("key length = %i" % len(alice_key))

  bob_sample = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0]
alice_sample = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0]
[1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
key length = 16
